In [1]:
import cv2
from darkflow.net.build import TFNet
import time
import matplotlib.pyplot as plt
import numpy as np

In [2]:
options={
    
    'model':'cfg/tiny-yolo-voc-1c.cfg',
    'load':875,
    'threshold':0.2
    
}

In [3]:
tfnet=TFNet(options)

Parsing cfg/tiny-yolo-voc-1c.cfg
Loading None ...
Finished in 0.0s

Building net ...
Source | Train? | Layer description                | Output size
-------+--------+----------------------------------+---------------
Instructions for updating:
Colocations handled automatically by placer.
       |        | input                            | (?, 416, 416, 3)
 Init  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 416, 416, 16)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 208, 208, 16)
 Init  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 208, 208, 32)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 104, 104, 32)
 Init  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 104, 104, 64)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 52, 52, 64)
 Init  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 52, 52, 128)
 Load  |  Yep!  | maxp 2x2p0_2                     | (?, 26, 26, 128)
 Init  |  Yep!  | conv 3x3p1_1  +bnorm  leaky      | (?, 26, 26, 256)
 Load 

OSError: File ./ckpt/tiny-yolo-voc-1c-875.meta does not exist.

In [4]:
colors=[tuple(255*np.random.rand(3)) for i in range(10)]

In [ ]:
capture=cv2.VideoCapture(0)

In [6]:
capture=cv2.VideoCapture(0)
capture.set(cv2.CAP_PROP_FRAME_WIDTH,1920)
capture.set(cv2.CAP_PROP_FRAME_HEIGHT,1080)
capture.set(cv2.CAP_PROP_FPS,20)
while True:
    stime=time.time()
    ret,frame=capture.read()
    results=tfnet.return_predict(frame)
    if ret:
        for color,result in zip(colors,results):
            t2=(result['bottomright']['x'],result['bottomright']['y'])
            t1=(result['topleft']['x'],result['topleft']['y'])
            label=result['label']
            confidence=result['confidence']
            
            text='{},{:.0F}%'.format(label,confidence*100)
            frame=cv2.rectangle(frame,t1,t2,color,6)
            frame=cv2.putText(frame,label,t1,cv2.FONT_HERSHEY_COMPLEX,1,(0,0,0),2)
        cv2.imshow('frame',frame)
    if cv2.waitKey(5) & 0xFF==27:
            break
    
capture.release()
cv2.destroyAllWindows()
        
            

NameError: name 'tfnet' is not defined

In [ ]:
import cv2
import numpy as np
from pynput.mouse import Controller,Button
import wx
mouse=Controller()
app=wx.App(False)
(sx,sy)=wx.GetDisplaySize()
(camx,camy)=(320,240)

lowerBound=np.array([20,100,100])
upperBound=np.array([30,255,255])

cam= cv2.VideoCapture(0)
cam.set(3,camx)
cam.set(4,camy)
kernelOpen=np.ones((5,5))
kernelClose=np.ones((20,20))

#font=cv2.cv.InitFont(cv2.cv.CV_FONT_HERSHEY_SIMPLEX,2,0.5,0,3,1)

while True:
    ret, img=cam.read()
    #img=cv2.resize(img,(340,220))

    #convert BGR to HSV
    imgHSV= cv2.cvtColor(img,cv2.COLOR_BGR2HSV)
    # create the Mask
    mask=cv2.inRange(imgHSV,lowerBound,upperBound)
    #morphology
    maskOpen=cv2.morphologyEx(mask,cv2.MORPH_OPEN,kernelOpen)
    maskClose=cv2.morphologyEx(maskOpen,cv2.MORPH_CLOSE,kernelClose)

    maskFinal=maskClose
    _,conts,h=cv2.findContours(maskFinal.copy(),cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    
    if(len(conts)==2):
        x1,y1,w1,h1=cv2.boundingRect(conts[0])
        x2,y2,w2,h2=cv2.boundingRect(conts[1])
        cv2.rectangle(img,(x1,y1),(x1+h1,y1+w1),(255,0,0),2)
        cv2.rectangle(img,(x2,y2),(x2+h2,y2+h2),(255,0,0),2)
        cx1=(x1+w1)//2
        cy1=(y1+h1)//2
        # centre coordinate of the 2nd object
        cx2=(x2+w2)//2
        cy2=(y2+h2)//2
        # centre coordinate of the line connection both points
        cx=(cx1+cx2)//2
        cy=(cy1+cy2)//2
        # Drawing the line
        cv2.line(img, (cx1,cy1),(cx2,cy2),(255,0,0),2)
    
    cv2.imshow("cam",img)
    if cv2.waitKey(10) & 0xFF==27:
        break